In [5]:
import tensorflow as tf

In [78]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [7]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10
input_size_to_hidden = (input_width//(max_pool1_k*max_pool2_k)) * (input_height//(max_pool1_k*max_pool2_k)) * n_conv2

In [12]:
weights = {
    "wc1" : tf.Variable(tf.random_normal([conv1_k, conv1_k, input_channels, n_conv1])),
    "wc2" : tf.Variable(tf.random_normal([conv2_k, conv2_k, n_conv1, n_conv2])),
    "wh1" : tf.Variable(tf.random_normal([input_size_to_hidden, n_hidden])),
    "wo" : tf.Variable(tf.random_normal([n_hidden, n_out]))    
    
}

biases = {
    
    "bc1" : tf.Variable(tf.random_normal([n_conv1])),
    "bc2" : tf.Variable(tf.random_normal([n_conv2])),
    "bh1" : tf.Variable(tf.random_normal([n_hidden])),
    "bo"  : tf.Variable(tf.random_normal([n_out]))
    
}

In [9]:
def conv(x, weights, bias, strides = 1):
    out = tf.nn.conv2d(x, weights, padding="SAME", strides = [1, strides, strides, 1])
    out = tf.nn.bias_add(out, bias)
    out = tf.nn.relu(out)
    return out

def maxpooling(x, k = 2):
    return tf.nn.max_pool(x, padding = "SAME", ksize = [1, k, k, 1], strides = [1, k, k, 1])

In [13]:
def cnn(x, weights, biases, keep_prob):
    x = tf.reshape(x, shape = [-1 ,input_height, input_width, input_channels])
    conv1 = conv(x, weights['wc1'], biases['bc1'], stride_conv1)
    conv1_pool = maxpooling(conv1, max_pool1_k)
    
    conv2 = conv(conv1_pool, weights['wc2'], biases['bc2'], stride_conv2)
    conv2_pool = maxpooling(conv2, max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool, shape = [-1, input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input, weights['wh1']), biases['bh1'])
    hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
    hidden_output = tf.nn.dropout(hidden_output_before_dropout, keep_prob) 
   
    output = tf.add(tf.matmul(hidden_output, weights['wo']), biases['bo'])
    return output

In [14]:
x = tf.placeholder("float", [None, input_pixels])
y = tf.placeholder(tf.int32, [None, n_out])
keep_prob = tf.placeholder("float")
pred = cnn(x, weights, biases, keep_prob)

In [15]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels = y))

In [16]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost)

In [17]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [19]:
batch_size = 100
for i in range(25):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost,optimize], feed_dict={x:batch_x , y:batch_y, keep_prob:0.8})
        total_cost += c
    print(total_cost)

4334.76204570729
3804.5849511059932
3518.4143811931854
3074.2838199395323
3440.924797706795
2853.7176599087366
2600.7519508978803
2585.047548919883
2880.474225623324
2861.549781503547


KeyboardInterrupt: 

In [20]:
predictions = tf.argmax(pred, 1)
correct_labels = tf.argmax(y, 1)
correct_predictions = tf.equal(predictions, correct_labels)
predictions,correct_preds  = sess.run([predictions, correct_predictions], feed_dict={x:mnist.test.images,
                                              y:mnist.test.labels, keep_prob:1.0})
correct_preds.sum()

9836

# using keras for mnist

In [146]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from sklearn.preprocessing import OneHotEncoder
from keras.utils import np_utils

In [147]:
(x_train,y_train), (x_test,y_test) = mnist.load_data()

In [118]:
'''y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)
one = OneHotEncoder()
one.fit(y_train)
y_train = one.transform(y_train).toarray()
print(y_train.shape)
y_test = one.transform(y_test).toarray()
y_test.shape

(60000, 10)


(10000, 10)

In [148]:
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
#num_pixels = x_train.shape[1] * x_train.shape[2]

y_train.shape


(60000, 10)

In [149]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784


n_conv1 = 32
n_conv2 = 64
conv1_k = 5
conv2_k = 5

n_hidden = 1024
n_out = 10
x_train = x_train.reshape((x_train.shape[0],input_height,input_width,input_channels))
x_test = x_test.reshape((x_test.shape[0],input_height,input_width,input_channels))
x_train = x_train / 255
x_test = x_test / 255



In [150]:
model = Sequential()

model.add(Conv2D(n_conv1,conv1_k,input_shape=(input_width, input_height,1), padding = 'same', use_bias = True, activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(n_conv2,conv2_k, padding = 'same', use_bias = True, activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(128, activation = 'relu'))
#model.add(Dense(64, activation = 'relu'))
#model.add(Dense(32, activation = 'relu'))
model.add(Dense(n_out, activation = 'softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_43 (Conv2D)           (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 3136)              0         
_________________________________________________________________
dense_55 (Dense)             (None, 128)               401536    
__________

In [151]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [152]:
model.fit(x_train, y_train, epochs=10, batch_size = 200, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 95s 2ms/step - loss: 0.2183 - acc: 0.9331 - val_loss: 0.0610 - val_acc: 0.9808
Epoch 2/10
60000/60000 [==============================] - 91s 2ms/step - loss: 0.0572 - acc: 0.9822 - val_loss: 0.0389 - val_acc: 0.9878
Epoch 3/10
60000/60000 [==============================] - 90s 2ms/step - loss: 0.0406 - acc: 0.9875 - val_loss: 0.0305 - val_acc: 0.9901
Epoch 4/10
60000/60000 [==============================] - 100s 2ms/step - loss: 0.0299 - acc: 0.9905 - val_loss: 0.0290 - val_acc: 0.9900
Epoch 5/10
60000/60000 [==============================] - 97s 2ms/step - loss: 0.0258 - acc: 0.9918 - val_loss: 0.0289 - val_acc: 0.9908
Epoch 6/10
60000/60000 [==============================] - 96s 2ms/step - loss: 0.0202 - acc: 0.9933 - val_loss: 0.0217 - val_acc: 0.9930
Epoch 7/10
60000/60000 [==============================] - 93s 2ms/step - loss: 0.0166 - acc: 0.9947 - val_loss: 0.0242 - val_ac